# Парсинг данных с JSON файла

In [41]:
import json
import glob
import pandas as pd
from datetime import datetime
from IPython.display import display

## Парсинг

In [44]:
all_jsons = glob.glob(r"Data\*.json")
all_data = []

for file_path in all_jsons:
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            data['file_source'] = file_path 
            all_data.append(data)
    except Exception as e:
        print(f"Ошибка при загрузке файла {file_path}: {str(e)}")    

## Посмотрим количество файлов

In [47]:
len(all_data)

14

## Извлечение данных из файлов

In [76]:
# Функция для преобразования даты в стандартный формат
def parse_custom_date(day, month, time):
    month_map = {
        'января': '01', 'февраля': '02', 'марта': '03', 'апреля': '04',
        'мая': '05', 'июня': '06', 'июля': '07', 'августа': '08',
        'сентября': '09', 'октября': '10', 'ноября': '11', 'декабря': '12'
    }
    try:
        return datetime.strptime(f"{day} {month_map[month]} {datetime.now().year if int(month_map[month]) <= datetime.now().month else datetime.now().year - 1} {time}", 
                               "%d %m %Y %H:%M")
    except:
        return None


# Обработка данных
def process_data(data_list):
    articles = []
    
    for data in data_list:
        # Обработка статей/публикаций
        refs = data.get('refs', [])
        if refs is None:  # Добавляем проверку на None
            refs = []
            
        for ref in refs:
            if isinstance(ref, list) and len(ref) >= 3:  # Проверяем тип и длину
                text = ref[0] if len(ref) > 0 else ""
                date_info = ref[1] if (len(ref) > 1 and isinstance(ref[1], dict)) else {'day': '1', 'month': 'января', 'time': '00:00'}
                stats = ref[2] if (len(ref) > 2 and isinstance(ref[2], dict)) else {'rate': 0, 'views': 0}
                
                # Извлечение ключевых слов из текста
                keywords = []
                text_lower = text.lower()
                for word in ['киберспорт', 'хакатон', 'премия', 'искусственный интеллект', 'разработка', 'технологии']:
                    if word in text_lower:
                        keywords.append(word)
                
                # Определение типа контента
                content_type = 'новость'
                if 'хакатон' in text_lower:
                    content_type = 'хакатон'
                elif 'премия' in text_lower:
                    content_type = 'награждение'
                elif 'исследование' in text_lower:
                    content_type = 'исследование'
                
                try:
                    date = parse_custom_date(date_info.get('day', '1'), 
                                           date_info.get('month', 'января'), 
                                           date_info.get('time', '00:00'))
                except:
                    date = None
                
                articles.append({
                    'text': text[:200] + '...' if len(text) > 200 else text,
                    'full_text': text,
                    'date': date,
                    'rating': stats.get('rate', 0),
                    'views': stats.get('views', 0),
                    'keywords': ', '.join(keywords) if keywords else 'нет',
                    'content_type': content_type,
                    'source_file': data.get('file_source', 'Неизвестно')
                })
    
    return  pd.DataFrame(articles)

## Создание датасета


In [79]:
articles_df = process_data(all_data)

## Вывод результатов


In [96]:
# Анализ данных
print("╔════════════════════════════╗")
print("║    Статистика по статьям   ║")
print("╠════════════════════════════╣")
print(f"║ • Средний рейтинг: {articles_df['rating'].mean():.1f}{' ' * 4}║")
print(f"║ • Средние просмотры: {articles_df['views'].mean():.0f}{' ' * 1}║")
print(f"║ • Медианные просмотры: {articles_df['views'].median():.0f}║")
print("╚════════════════════════════╝")

╔════════════════════════════╗
║    Статистика по статьям   ║
╠════════════════════════════╣
║ • Средний рейтинг: 23.9    ║
║ • Средние просмотры: 21016 ║
║ • Медианные просмотры: 7000║
╚════════════════════════════╝


## Сделаем  файл из полученных данных

In [79]:
articles_df.to_csv('jsons_info.csv')